In [8]:
# import modules
import sys
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pdb

In [40]:
volumns = sections[2]

volumn_names = []
for v in volumns:
    volumn_names.append(v.string)
    
pres = pres[:2]
links = []
for pre in pres:
    for link in pre.findAll('a',href=True):
        #pdb.set_trace()
        index = link['href'].split('-')[-1].split('.')[0]
        links.append("https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-" + index + ".htm")
        
print('links: ')

index = -1
for i in range(len(links)):
    link = links[i]

    if link[-5] == "a":
        index = i
        break;
if index != -1:
    links.pop(index)

links.pop(-1)
i = 1
for link in links:
    print(i, link)
    i = i + 1

links: 
1 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-19251201.htm
2 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-192703.htm
3 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-19281005.htm
4 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-19281125.htm
5 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-192912.htm
6 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-19300105.htm
7 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-193005.htm
8 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-19330812.htm
9 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-193310.htm
10 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-193401.htm
11 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-19340127.htm
12 https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-19351227.htm
13 https://www.

In [67]:
def get_one_link(weblink):
    
    file = open("1.txt","w",encoding= 'utf-8')
    
    html = urlopen(weblink)
    #bsObj = BeautifulSoup(html.read(),"lxml")
    bsObj = BeautifulSoup(html.read(),"html5lib")
    # 
    # get title
    title = bsObj.find("p", {"class": "title1"})
    if title is not None:
        title = title.get_text()
        # get date
        date = bsObj.find("p", {"class": "date"})
        if date is not None:
            date = date.get_text()
            # add title and date info to article
            title_date =  "\\section*{\\myformat{" + title + "}\\\\" + "\\myformat{" + date + "}}" \
                       + "\\addcontentsline{toc}{section}{" + title + "}"
        else:
            title_date =  "\\section*{\\myformat{" + title + "}\\\\" \
                       + "\\addcontentsline{toc}{section}{" + title + "}"
    
        file.write(str(title_date.encode('utf-8').decode('utf-8'))+"\n")
    else:
        title_date = bsObj.find('title').get_text()
        
        if '（' in title_date:
            alist = title_date.split('（')
            title = alist[0]
            date = '（' + alist[1]
            title_date =  "\\section*{\\myformat{" + title + "}\\\\" + "\\myformat{" + date + "}}" \
                       + "\\addcontentsline{toc}{section}{" + title + "}"
    
            file.write(str(title_date.encode('utf-8').decode('utf-8'))+"\n")
        elif "—" in title_date:
            alist = title_date.split('—')
            title = alist[0]
            date = '---' + alist[-1]
            title_date =  "\\section*{\\myformat{" + title + "}\\\\" + "\\myformat{" + date + "}}" \
                       + "\\addcontentsline{toc}{section}{" + title + "}"
        else:
            pass
    print(title)
    # get blockquote
    blockquote = bsObj.find("blockquote")
    #pdb.set_trace()
    #\u3000 是全角的空白符
    is_there_quote = True
    
    if len(bsObj.findAll("p"))>2 and bsObj.findAll("p")[1].find_next().name == 'p':
        ps = bsObj.findAll("p")
        blockquote = ps[2].get_text()
    elif blockquote:
        blockquote = blockquote.get_text()
    else:
        is_there_quote = False
    if is_there_quote:
        while blockquote[0] == "\u3000" or blockquote[0] == "*":
            blockquote = blockquote[1:]
        # add quote to article
        quote_data = "\\begin{introduction}" + "\\item " + blockquote + "\\end{introduction}"
        file.write(str(quote_data.encode('utf-8').decode('utf-8'))+"\n")
    
    # get references
    
    references = bsObj.findAll('a')
    last_ref = references[-1].find('sup')
    if last_ref is None:
        last_ref = references[-1].string
    else:
        last_ref = last_ref.get_text()
    
    if last_ref[1:-1].isdigit():
        num_refs = int(last_ref[1:-1])
        references = references[-num_refs:]
    
        ref_context = []
        for ref in references:
            #print(ref)
            ref_context.append(ref.next_sibling)
        references = ref_context
    
    # get content
    body = bsObj.find('body')
    content = body.get_text()
    paragraphs = content.split('\n')
    
    # locate the first paragraph after quote and the last paragraph before "注释"
    index2 = len(paragraphs)-1
    #pdb.set_trace()
    for i in range(len(paragraphs)):
        par = paragraphs[i]
        if is_there_quote:
            if par.find(blockquote) != -1 or par.find("这个十卷本的集子在一九七六年由香港的一山书店翻印")!=-1:
                index1 = i+1
        else:
            if par.find(date) != -1:
                index1 = i+1
            elif par.find(title) != -1:
                index1 = i+ 1     
            else:
                pass
        
        #if par.find("注释") != -1 or par.find("注\u3000释") != -1:
        if par == "注释" or par.find("注\u3000释") != -1:
            index2 = i-1
    #pdb.set_trace()       
    if body.find('h3') is None and body.find('h4') is None: # no subsections/h3 or h4
        flag = True
        for i in range(index1, index2+1):
            line = str(paragraphs[i].encode('utf-8').decode('utf-8')) 
            if line == "" or line == "\r":
                continue
            if flag:
                #pdb.set_trace()
                while line[0] == "\u3000":
                    line = line[1:]
                flag = False
            file.write(line +"\\\\"+"\n")
        file.close()
        
    else: # subsections/h3 and subsubsections/h4
        
        
        h3 = bsObj.findAll("h3")
        h3_names = [h.get_text() for h in h3]
        h4 = bsObj.findAll("h4")
        h4_names = [h.get_text() for h in h4]
        # find h4 behind h3
        h34 = bsObj.findAll({"h3","h4"})
        flag = True
        for i in range(index1, index2+1):
            line = str(paragraphs[i].encode('utf-8').decode('utf-8')) 
            if line == "" or line == "\r":
                continue
            
            # deal with h3 
            #if is_h34(line, h3_names):
            if line in h3_names:
                # add h3 info
                h3_date = "\\subsection*{\\myformat{" + line + "}}"
                file.write(str(h3_date.encode('utf-8').decode('utf-8'))+"\n")
                flag = True
            # deal with h4
            #elif is_h34(line, h4_names):
            elif line in h4_names:
                # add h3 info
                h4_date = "\\subsubsection*{\\myformat{" + line + "}}"
                file.write(str(h4_date.encode('utf-8').decode('utf-8'))+"\n")
                flag = True
            # deal with regular content
            else:
                if flag:
                    #pdb.set_trace()
                    while len(line)>0 and line[0] == "\u3000":
                        line = line[1:]
                    flag = False
                file.write(line +"\\\\"+"\n")
        file.close()
            
        
        
        
        
        

    # change quotes
    file = open("1.txt", "r", encoding="utf-8")
    latex = []
    ref_index = 1
    for line in file:
        for ref_index in range(1, len(references)+1):
            old = '[' + str(ref_index)+']';
            old = str(old.encode('utf-8').decode('utf-8'))
            if line.find(old) != -1:
                new = "\\footnote" + old + "{" + references[ref_index-1] + "}"
                new = str(new.encode('utf-8').decode('utf-8')) 
                line = line.replace(old, new)
      
        latex.append(line)
    
    file.close()
    
    file = open("2.txt", "a",  encoding="utf-8")
    for line in latex:
        file.write(line)
    # new line
    line = "\\newpage"
    line = str(line.encode('utf-8').decode('utf-8'))
    file.write(line)
    file.close()
            
        

# Volume 2

In [43]:
volumns = sections[2]

pres = bsObj.findAll('pre')

pres_v2 = pres[2]

v2 = pres_v2.findAll('a')

v2_titles = []
v2_links = []

for a in v2:
    v2_titles.append(a.string)
    #index = ''.join(c for c in a['href'] if c.isdigit())
    index = a['href'].split('-')[-1].split('.')[0]
    v2_links.append("https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-"+index+".htm")

# delete 2.txt

import os
os.remove("2.txt")
print("2.txt Removed!")
# write chapter 1 

file = open("2.txt", "a",  encoding="utf-8")
line = "\\chapter*{抗日战争时期（上）}\\addcontentsline{toc}{chapter}{抗日战争时期（上）}"
line = str(line.encode('utf-8').decode('utf-8'))
file.write(line)
line = "\\newpage"
line = str(line.encode('utf-8').decode('utf-8'))
file.write(line)
file.close()

i = 1


for link in v2_links:
    #pdb.set_trace()
    get_one_link(link)
    print(i, "is done!")
    i += 1



2.txt Removed!
反对日本进攻的方针、办法和前途
1 is done!
为动员一切力量争取抗战胜利而斗争
2 is done!
反对自由主义
3 is done!
国共合作成立后的迫切任务
4 is done!
和英国记者贝特兰的谈话
5 is done!
上海太原失陷以后抗日战争的形势和任务
6 is done!
陕甘宁边区政府、第八路军后方留守处布告
7 is done!
抗日游击战争的战略问题
8 is done!
论持久战
9 is done!
毛泽东：中国共产党在民族战争中的地位
10 is done!
毛泽东：抗日民族战争与抗日民族统一战线发展的新阶段
11 is done!
统一战线中的独立自主问题
12 is done!
战争和战略问题
13 is done!
五四运动
14 is done!
青年运动的方向
15 is done!
反对投降活动
16 is done!
必须制裁反动派
17 is done!
关于国际新形势对新华日报[1]记者的谈话
18 is done!
和中央社、扫荡报、新民报[1]三记者的谈话
19 is done!
苏联利益和人类利益的一致
20 is done!
《共产党人》发刊词
21 is done!
目前形势和党的任务
22 is done!
大量吸收知识分子
23 is done!
中国革命和中国共产党
24 is done!
中国革命与中国共产党
25 is done!
斯大林是中国人民的朋友
26 is done!
纪念白求恩
27 is done!
新民主主义论
28 is done!
克服投降危险，力争时局好转
29 is done!
团结一切抗日力量，反对反共顽固派
30 is done!
向国民党的十点要求
31 is done!
延安民众讨汪拥蒋大会通电
32 is done!
《中国工人》发刊词
33 is done!
必须强调团结和进步
34 is done!
中共中央关于目前时局与党的任务的决定
35 is done!
新民主主义的宪政
36 is done!
抗日根据地的政权问题
37 is done!
目前抗日统一战线中的策略问题
38 is done!
放手发展抗日力量，抵抗反共顽固派的进攻
39 is done!
团结到底
40 is done!
论政策
41 is done

In [45]:
import os
os.remove("2.txt")
print("2.txt Removed!")
link = "https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-19400201baa.htm"
get_one_link(link)

2.txt Removed!
延安民众讨汪拥蒋大会通电


# Volume 3

In [51]:
volumns = sections[3]

pres = bsObj.findAll('pre')

pres_v3 = pres[3]

v3 = pres_v3.findAll('a')

v3_titles = []
v3_links = []

for a in v3:
    v3_titles.append(a.string)
    #index = ''.join(c for c in a['href'] if c.isdigit())
    index = a['href'].split('-')[-1].split('.')[0]
    v3_links.append("https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-"+index+".htm")


# delete 2.txt

import os
os.remove("2.txt")
print("2.txt Removed!")
# write chapter 1 

file = open("2.txt", "a",  encoding="utf-8")
line = "\\chapter*{抗日战争时期（下）}\\addcontentsline{toc}{chapter}{抗日战争时期（下）}"
line = str(line.encode('utf-8').decode('utf-8'))
file.write(line)
line = "\\newpage"
line = str(line.encode('utf-8').decode('utf-8'))
file.write(line)
file.close()

i = 1

for link in v3_links:
    #pdb.set_trace()
    get_one_link(link)
    print(i, "is done!")
    i += 1



2.txt Removed!
《农村调查》的序言和跋
1 is done!
改造我们的学习
2 is done!
揭破远东慕尼黑的阴谋
3 is done!
关于反法西斯的国际统一战线
4 is done!
在陕甘宁边区参议会的演说
5 is done!
整顿党的作风
6 is done!
反对党八股
7 is done!
在延安文艺座谈会上的讲话
8 is done!
一个极其重要的政策
9 is done!
第二次世界大战的转折点
10 is done!
祝十月革命二十五周年
11 is done!
抗日时期的经济问题和财政问题
12 is done!
关于领导方法的若干问题
13 is done!
质问国民党
14 is done!
开展根据地的减租、生产和拥政爱民运动
15 is done!
评国民党十一中全会和三届二次国民参政会
16 is done!
组织起来
17 is done!
学习和时局
18 is done!
为人民服务
19 is done!
评蒋介石在双十节的演说
20 is done!
文化工作中的统一战线
21 is done!
必须学会做经济工作
22 is done!
游击区也能够进行生产
23 is done!
两个中国之命运
24 is done!
论联合政府
25 is done!
论联合政府
26 is done!
愚公移山
27 is done!
论军队生产自给，兼论整风和生产两大运动的重要性
28 is done!
赫尔利和蒋介石的双簧已经破产
29 is done!
评赫尔利政策的危险
30 is done!
给福斯特同志的电报
31 is done!
对日寇的最后一战
32 is done!


# Volume 4

In [61]:
volumns = sections[4]

pres = bsObj.findAll('pre')

pres_v4 = pres[4]

v4 = pres_v4.findAll('a')

v4_titles = []
v4_links = []

for a in v4:
    v4_titles.append(a.string)
    #index = ''.join(c for c in a['href'] if c.isdigit())
    index = a['href'].split('-')[-1].split('.')[0]
    v4_links.append("https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-"+index+".htm")


# delete 2.txt

import os
os.remove("2.txt")
print("2.txt Removed!")
# write chapter 1 

file = open("2.txt", "a",  encoding="utf-8")
line = "\\chapter*{第三次国内革命战争时期}\\addcontentsline{toc}{chapter}{第三次国内革命战争时期}"
line = str(line.encode('utf-8').decode('utf-8'))
file.write(line)
line = "\\newpage"
line = str(line.encode('utf-8').decode('utf-8'))
file.write(line)
file.close()

i = 1

#get_one_link(v4_links[60])

for link in v4_links:
    #pdb.set_trace()
    get_one_link(link)
    print(i, "is done!")
    i += 1



2.txt Removed!
抗日战争胜利后的时局和我们的方针
1 is done!
蒋介石在挑动内战
2 is done!
第十八集团军总司令给蒋介石的两个电报
3 is done!
评蒋介石发言人谈话
4 is done!
中共中央关于同国民党进行和平谈判的通知
5 is done!
关于重庆谈判
6 is done!
国民党进攻的真相
7 is done!
减租和生产是保卫解放区的两件大事
8 is done!
一九四六年解放区工作的方针
9 is done!
建立巩固的东北根据地
10 is done!
关于目前国际形势的几点估计
11 is done!
以自卫战争粉碎蒋介石的进攻
12 is done!
和美国记者安娜·刘易斯·斯特朗[1]的谈话
13 is done!
集中优势兵力，各个歼灭敌人
14 is done!
美国“调解”真相和中国内战前途
15 is done!
三个月总结
16 is done!
迎接中国革命的新高潮
17 is done!
中共中央关于暂时放弃延安和保卫陕甘宁边区的两个文件
18 is done!
关于西北战场的作战方针
19 is done!
蒋介石政府已处在全民的包围中
20 is done!
解放战争第二年的战略方针
21 is done!
中国人民解放军宣言
22 is done!
中国人民解放军总部关于重行颁布三大纪律八项注意的训令
23 is done!
目前形势和我们的任务
24 is done!
关于建立报告制度
25 is done!
关于目前党的政策中的几个重要问题
26 is done!
军队内部的民主运动
27 is done!
在不同地区实施土地法的不同策略
28 is done!
纠正土地改革宣传中的“左”倾错误
29 is done!
新解放区土地改革要点
30 is done!
关于工商业政策
31 is done!
关于民族资产阶级和开明绅士问题
32 is done!
评西北大捷兼论解放军的新式整军运动
33 is done!
关于情况的通报
34 is done!
在晋绥干部会议上的讲话
35 is done!
对晋绥日报编辑人员的谈话
36 is done!
再克洛阳后给洛阳前线指挥部的电报
37 is done!
新解放区农村工作的策略问题
38 is done!
一九四八年的土地

# Volume 5

In [76]:
volumns = sections[5]

pres = bsObj.findAll('pre')

pres_v5 = pres[5]

v5 = pres_v5.findAll('a')

v5_titles = []
v5_links = []

for a in v5:
    v5_titles.append(a.string)
    #index = ''.join(c for c in a['href'] if c.isdigit())
    index = a['href'].split('-')[-1].split('.')[0]
    v5_links.append("https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-"+index+".htm")


# delete 2.txt

import os
os.remove("2.txt")
print("2.txt Removed!")
# write chapter 1 

file = open("2.txt", "a",  encoding="utf-8")
line = "\\chapter*{社会主义革命和社会主义建设时期（一）}\\addcontentsline{toc}{chapter}{社会主义革命和社会主义建设时期（一）}"
line = str(line.encode('utf-8').decode('utf-8'))
file.write(line)
line = "\\newpage"
line = str(line.encode('utf-8').decode('utf-8'))
file.write(line)
file.close()

i = 1

#get_one_link(v5_links[47])

for link in v5_links:
    #pdb.set_trace()
    if i == 1 or i == 47 or i==49: 
        print(v5_titles[i-1])
        print(i, "skip")
        i += 1
        continue
    get_one_link_v5(link)
    print(i, "is done!")
    i += 1



2.txt Removed!
出版说明
1 skip
中国人民站起来了
2 is done!
中国人民大团结万岁
3 is done!
人民英雄们永垂不朽
4 is done!
永远保持艰苦奋斗的作风
5 is done!
征询对待富农策略问题的意见
6 is done!
为争取国家财政经济状况的基本好转而斗争
7 is done!
不要四面出击
8 is done!
做一个完全的革命派
9 is done!
你们是全民族的模范人物
10 is done!
给中国人民志愿军的命令
11 is done!
中国人民志愿军要爱护朝鲜的一山一水一草一木
12 is done!
中共中央政治局扩大会议决议要点
13 is done!
镇压反革命必须实行党的群众路线
14 is done!
镇压反革命必须打得稳，打得准，打得狠
15 is done!
应当重视电影《武训传》的讨论
16 is done!
三大运动的伟大胜利
17 is done!
关于“三反”、“五反”的斗争
18 is done!
把农业互助合作当作一件大事去做
19 is done!
元旦祝词
20 is done!
中共中央关于西藏工作方针的指示
21 is done!
工人阶级与资产阶级的矛盾是国内的主要矛盾
22 is done!
团结起来，划清敌我界限
23 is done!
祝贺中国人民志愿军的重大胜利
24 is done!
反对官僚主义、命令主义和违法乱纪
25 is done!
批判大汉族主义
26 is done!
解决“五多”问题
27 is done!
对刘少奇、杨尚昆破坏纪律擅自以中央名义发出文件的批评
28 is done!
批判离开总路线的右倾观点
29 is done!
青年团的工作要照顾青年的特点
30 is done!
关于国家资本主义
31 is done!
党在过渡时期的总路线
32 is done!
反对党内的资产阶级思想
33 is done!
改造资本主义工商业的必经之路
34 is done!
抗美援朝的伟大胜利和今后的任务
35 is done!
批判梁漱溟的反动思想
36 is done!
关于农业互助合作的两次谈话
37 is done!
关于中华人民共和国宪法草案
38 is done!
为建设一个伟大的社会主义国家而奋斗
39 is done!
关于

In [73]:
def get_one_link_v5(weblink):
    
    file = open("1.txt","w",encoding= 'utf-8')
    
    html = urlopen(weblink)
    #bsObj = BeautifulSoup(html.read(),"lxml")
    bsObj = BeautifulSoup(html.read(),"html5lib")
    # 
    # get title
    title = bsObj.find("p", {"class": "title1"})
    if title is not None:
        title = title.get_text()
        # get date
        date = bsObj.find("p", {"class": "date"})
        if date is not None:
            date = date.get_text()
            # add title and date info to article
            title_date =  "\\section*{\\myformat{" + title + "}\\\\" + "\\myformat{" + date + "}}" \
                       + "\\addcontentsline{toc}{section}{" + title + "}"
        else:
            title_date =  "\\section*{\\myformat{" + title + "}\\\\" \
                       + "\\addcontentsline{toc}{section}{" + title + "}"
    
        file.write(str(title_date.encode('utf-8').decode('utf-8'))+"\n")
    else:
        title_date = bsObj.find('title').get_text()
        
        if '（' in title_date:
            alist = title_date.split('（')
            title = alist[0]
            date = '（' + alist[1]
            title_date =  "\\section*{\\myformat{" + title + "}\\\\" + "\\myformat{" + date + "}}" \
                       + "\\addcontentsline{toc}{section}{" + title + "}"
    
            file.write(str(title_date.encode('utf-8').decode('utf-8'))+"\n")
        elif "—" in title_date:
            alist = title_date.split('—')
            title = alist[0]
            date = '---' + alist[-1]
            title_date =  "\\section*{\\myformat{" + title + "}\\\\" + "\\myformat{" + date + "}}" \
                       + "\\addcontentsline{toc}{section}{" + title + "}"
        else:
            pass
    print(title)
    # get blockquote
    blockquote = bsObj.find("blockquote")
    #pdb.set_trace()
    #\u3000 是全角的空白符
    is_there_quote = True
    
    if len(bsObj.findAll("p"))>2 and bsObj.findAll("p")[1].find_next().name == 'p':
        ps = bsObj.findAll("p")
        blockquote = ps[2].get_text()
    elif blockquote:
        blockquote = blockquote.get_text()
    else:
        is_there_quote = False
    if is_there_quote:
        while blockquote[0] == "\u3000" or blockquote[0] == "*":
            blockquote = blockquote[1:]
        # add quote to article
        quote_data = "\\begin{introduction}" + "\\item " + blockquote + "\\end{introduction}"
        file.write(str(quote_data.encode('utf-8').decode('utf-8'))+"\n")
    
    # get references
    
    references = bsObj.findAll('a')
    last_ref = references[-1].find('sup')
    if last_ref is None:
        last_ref = references[-1].string
    else:
        last_ref = last_ref.get_text()
    
    if last_ref[1:-1].isdigit():
        num_refs = int(last_ref[1:-1])
        references = references[-num_refs:]
    
        ref_context = []
        for ref in references:
            #print(ref)
            ref_context.append(ref.next_sibling)
        references = ref_context
    
    # get content
    body = bsObj.find('body')
    content = body.get_text()
    paragraphs = content.split('\n')
    
    # locate the first paragraph after quote and the last paragraph before "注释"
    index2 = len(paragraphs)-1
    #pdb.set_trace()
    for i in range(len(paragraphs)):
        par = paragraphs[i]
        if is_there_quote:
            if par.find(blockquote) != -1 or par.find("这个十卷本的集子在一九七六年由香港的一山书店翻印")!=-1:
                index1 = i+1
        else:
            if par.find(date) != -1:
                index1 = i+1
            elif par.find(title) != -1:
                index1 = i+ 1     
            else:
                pass
        
        #if par.find("注释") != -1 or par.find("注\u3000释") != -1:
        if par == "注释" or par.find("注\u3000释") != -1:
            index2 = i-1
    #pdb.set_trace()       
    if body.find('h3') is None and body.find('h4') is None: # no subsections/h3 or h4
        flag = True
        for i in range(index1, index2+1):
            line = str(paragraphs[i].encode('utf-8').decode('utf-8')) 
            if line == "" or line == "\r":
                continue
            if flag:
                #pdb.set_trace()
                while line[0] == "\u3000":
                    line = line[1:]
                flag = False
            if line.find("《毛泽东选集》第五卷")!= -1:
                line = "\\begin{flushright}" + line + "\\end{flushright}"
                file.write(line + "\n")
            else:
                file.write(line +"\\\\"+"\n")
        file.close()
        
    else: # subsections/h3 and subsubsections/h4
        
        
        h3 = bsObj.findAll("h3")
        h3_names = [h.get_text() for h in h3]
        h4 = bsObj.findAll("h4")
        h4_names = [h.get_text() for h in h4]
        # find h4 behind h3
        h34 = bsObj.findAll({"h3","h4"})
        flag = True
        for i in range(index1, index2+1):
            line = str(paragraphs[i].encode('utf-8').decode('utf-8')) 
            if line == "" or line == "\r":
                continue
            
            # deal with h3 
            #if is_h34(line, h3_names):
            if line in h3_names:
                # add h3 info
                h3_date = "\\subsection*{\\myformat{" + line + "}}"
                file.write(str(h3_date.encode('utf-8').decode('utf-8'))+"\n")
                flag = True
            # deal with h4
            #elif is_h34(line, h4_names):
            elif line in h4_names:
                # add h3 info
                h4_date = "\\subsubsection*{\\myformat{" + line + "}}"
                file.write(str(h4_date.encode('utf-8').decode('utf-8'))+"\n")
                flag = True
            # deal with regular content
            else:
                if flag:
                    #pdb.set_trace()
                    while len(line)>0 and line[0] == "\u3000":
                        line = line[1:]
                    flag = False
                if line.find("《毛泽东选集》第五卷")!= -1:
                    line = "\\begin{flushright}" + line + "\\end{flushright}"
                    file.write(line + "\n")
                else:
                    file.write(line +"\\\\"+"\n")
        file.close()
            
        
        
        
        
        

    # change quotes
    file = open("1.txt", "r", encoding="utf-8")
    latex = []
    ref_index = 1
    for line in file:
        for ref_index in range(1, len(references)+1):
            old = '[' + str(ref_index)+']';
            old = str(old.encode('utf-8').decode('utf-8'))
            if line.find(old) != -1:
                new = "\\footnote" + old + "{" + references[ref_index-1] + "}"
                new = str(new.encode('utf-8').decode('utf-8')) 
                line = line.replace(old, new)
      
        latex.append(line)
    
    file.close()
    
    file = open("2.txt", "a",  encoding="utf-8")
    for line in latex:
        file.write(line)
    # new line
    line = "\\newpage"
    line = str(line.encode('utf-8').decode('utf-8'))
    file.write(line)
    file.close()
            
        

In [84]:
#
link = v5_links[48]
html = urlopen(link)
bsObj = BeautifulSoup(html.read(),"html5lib")

h4 = bsObj.findAll("h4")
h4_names = [h.get_text() for h in h4]
h3 = bsObj.findAll("h3")
h3_names = [h.get_text() for h in h3]
body = bsObj.find('body')
content = body.get_text()
paragraphs = content.split('\n')
#pdb.set_trace()

h34 = bsObj.findAll({"h3","h4"})
flag = True
index1 = 7
index2 = 143
file = open("5.txt", "w")
for i in range(index1, index2+1):
    line = str(paragraphs[i].encode('utf-8').decode('utf-8')) 
    if line == "" or line == "\r":
        continue
            
    # deal with h3 
    #if is_h34(line, h3_names):
    if line in h3_names:
        # add h3 info
        h3_date = "\\subsection*{\\myformat{" + line + "}}"
        file.write(str(h3_date.encode('utf-8').decode('utf-8'))+"\n")
        flag = True
    # deal with h4
    #elif is_h34(line, h4_names):
    elif line in h4_names:
        # add h3 info
        h4_date = "\\subsubsection*{\\myformat{" + line + "}}"
        file.write(str(h4_date.encode('utf-8').decode('utf-8'))+"\n")
        flag = True
    # deal with regular content
    else:
        if flag:
            #pdb.set_trace()
            while len(line)>0 and line[0] == "\u3000":
                line = line[1:]
            flag = False
        if line.find("一文按语")!= -1 or line.find("《毛泽东选集》第五卷")!=-1:
            line = "\\begin{flushright}" + line + "\\end{flushright}"
            file.write(line + "\n")
        else:
            file.write(line +"\\\\"+"\n")
file.close()
            